In [1]:
!pip install python-dotenv
!pip install -U torch transformers

  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)


# Gemma Model

In [8]:
import torch
import os
from transformers import AutoTokenizer, AutoModelForCausalLM
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()
ACCESS_TOKEN = os.getenv("ACCESS_TOKEN")  # Reads .env file with ACCESS_TOKEN=<your hugging face access token>

# Model ID and Tokenizer
model_id = "google/gemma-2b-it"
tokenizer = AutoTokenizer.from_pretrained(model_id, token=ACCESS_TOKEN)

# Load the model for CPU (no quantization for CPU)
model = AutoModelForCausalLM.from_pretrained(model_id, token=ACCESS_TOKEN)
model.eval()

# Set device to CPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)  # Ensure the model is on the correct device

print(f"Using {device}")

# Example code to test model (optional)
# input_text = "Hello, how are you?"
# inputs = tokenizer(input_text, return_tensors="pt")
# outputs = model.generate(**inputs)
# print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:23<00:00, 11.62s/it]


Using cpu


In [9]:
def inference(question: str, context: str):

    if context == None or context == "":
        prompt = f"""Give a detailed answer to the following question. Question: {question}"""
    else:
        prompt = f"""Using the information contained in the context, give a detailed answer to the question.
            Context: {context}.
            Question: {question}"""
    chat = [
        {"role": "user", "content": prompt},
        # { "role": "model", "content": "Recurrent Attention (RAG)** is a novel neural network architecture specifically designed" }
    ]
    formatted_prompt = tokenizer.apply_chat_template(
        chat,
        tokenize=False,
        add_generation_prompt=True,
    )
    inputs = tokenizer.encode(
        formatted_prompt, add_special_tokens=False, return_tensors="pt"
    ).to(device)
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs,
            max_new_tokens=250,
            do_sample=False,
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=False)
    response = response[len(formatted_prompt) :]  # remove input prompt from reponse
    response = response.replace("<eos>", "")  # remove eos token
    return response


question = "What is a transformer?"
print(inference(question=question, context=""))

Sure, here's a detailed answer to the question:

**What is a Transformer?**

A transformer is an electrical device that transfers energy from one circuit to another through an alternating current (AC) current. It consists of two or more coils of wire wound around a core of ferromagnetic material, such as iron or silicon. When an alternating current is passed through the primary coil, it creates a magnetic field. This magnetic field then induces an alternating current in the secondary coil, even if the coils are separated by a gap.

**How it works:**

The transformer works based on the principle of electromagnetic induction. When an alternating current is induced in the secondary coil, it creates a magnetic field that opposes the original magnetic field in the primary coil. This induced magnetic field then causes current to flow in the secondary coil, even though the coils are separated by a gap.

The turns ratio of a transformer determines the ratio of the number of turns in the primar

# Document Loading and Splitting

In [10]:
!pip install pypdf
!pip install tiktoken

  Using cached pypdf-4.3.1-py3-none-any.whl.metadata (7.4 kB)
Using cached pypdf-4.3.1-py3-none-any.whl (295 kB)
  Using cached tiktoken-0.7.0-cp312-cp312-win_amd64.whl.metadata (6.8 kB)
Using cached tiktoken-0.7.0-cp312-cp312-win_amd64.whl (799 kB)


In [11]:
from langchain.document_loaders import PyPDFLoader

loaders = [
    PyPDFLoader(r'C:\Users\Ammar Abdulhussain\Desktop\Chatbot\SRB_MPSTME_2023-24_jzLuNN33kE.pdf'),
    PyPDFLoader(r'C:\Users\Ammar Abdulhussain\Desktop\Chatbot\Ammar_A026_CF-1_B2.pdf'),
]
pages = []
for loader in loaders:
    pages.extend(loader.load())


In [12]:
!pip install langchain


In [13]:
!pip install -U langchain-community

In [14]:
from langchain.text_splitter import TokenTextSplitter

text_splitter = TokenTextSplitter(chunk_size=128, chunk_overlap=12)
docs = text_splitter.split_documents(pages)

In [15]:
print(docs[0].page_content)

   
  
 
 
 
 
 
 
 
 
 
 
 
 
STUDENT RESOURCE BO OK 
 (2023-24) 
Part -I 
 
 
 
 
NMIMS (Deemed -to-be) 
UNIVERSITY  
  



# Embeddings and Vector Store

In [16]:
!pip install -U sentence-transformers

In [19]:

import numpy as np
from langchain_community.embeddings import (
    HuggingFaceEmbeddings
)
encoder = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L12-v2', model_kwargs = {'device': "cpu"})
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L12-v2", clean_up_tokenization_spaces=True)


In [20]:
embeddings1 = encoder.embed_query("RAG")
embeddings2 = encoder.embed_query(docs[0].page_content)
print(np.dot(embeddings1, embeddings2))

0.07636687992454526


In [21]:
!pip install faiss-cpu

   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ---- ----------------------------------- 1.6/14.6 MB 9.3 MB/s eta 0:00:02
   --------- ------------------------------ 3.4/14.6 MB 9.6 MB/s eta 0:00:02
   ---------------- ----------------------- 6.0/14.6 MB 10.5 MB/s eta 0:00:01
   ---------------------- ----------------- 8.4/14.6 MB 10.8 MB/s eta 0:00:01
   --------------------------- ------------ 10.0/14.6 MB 10.2 MB/s eta 0:00:01
   ---------------------------- ----------- 10.5/14.6 MB 9.1 MB/s eta 0:00:01
   ----------------------------- ---------- 10.7/14.6 MB 8.4 MB/s eta 0:00:01
   ------------------------------ --------- 11.3/14.6 MB 7.3 MB/s eta 0:00:01
   --------------------------------- ------ 12.1/14.6 MB 6.6 MB/s eta 0:00:01
   ---------------------------------- ----- 12.6/14.6 MB 6.3 MB/s eta 0:00:01
   ------------------------------------ --- 13.4/14.6 MB 5.9 MB/s eta 0:00:01
   -------------------------------------- - 14.2/14.6 MB 5.8 MB/s eta 0:0

In [22]:
from langchain.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy
faiss_db = FAISS.from_documents(docs, encoder, distance_strategy=DistanceStrategy.DOT_PRODUCT)

In [26]:
question = "Tell me about Sony PlayStation Network breach (2011)"
retrieved_docs = faiss_db.similarity_search(question, k=5)
context = "".join(doc.page_content + "\n" for doc in retrieved_docs)
print(context)

  exposed personal 
information of millions of users. Digital forensics were used to analyze the 
compromised servers, identify the type of malware used in the attack, and 
understand how the attackers gained access. This information was crucial for 
improving PlayStation Network security and potentially identifying the culprits.  
3. Network Forensics:  
• Real World Example:  The Mirai botnet attack (2016)  involved a massive 
network of hacked devices launching a DDoS attack against major websites. 
Network forensics helped investigators track the origin of the attack traffic back 
SVKM’S NMIMS (Deemed -to-be University)  
MUKESH PATEL SCHOOL OF TECHNOLOGY MANAGEMENT AND ENGINEERING  
NAVI MUMBAI CAMPUS  
 
deleted text messages exchanged between the victims and the suspect ("Andy") 
which, along with other evidence, led to the arrest of Philip Markoff.  
2. Cyber Forensics (Digital Forensics):  
• Example:  The Sony PlayStation Network breach (2011)  exposed personal 
information o

In [27]:
torch.cuda.empty_cache()
print(inference(question=question, context=context))

ValueError: Cannot use apply_chat_template() because tokenizer.chat_template is not set and no template argument was passed! For information about writing templates and setting the tokenizer.chat_template attribute, please see the documentation at https://huggingface.co/docs/transformers/main/en/chat_templating

In [ ]:
print("For this answer I used the following documents:")
for doc in retrieved_docs:
    print(doc.metadata)